# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd

In [2]:
ttt = pd.read_csv('tic-tac-toe.csv')

In [3]:
ttt.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [4]:
ttt = ttt.astype('category')

In [5]:
#unused
#ttt_dummies= pd.get_dummies(ttt.columns, prefix='pos')

In [6]:
ttt_dummies

,pos_BL,pos_BM,pos_BR,pos_ML,pos_MM,pos_MR,pos_TL,pos_TM,pos_TR,pos_class
0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1


In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
cols = ttt.columns
cols

Index(['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class'], dtype='object')

In [9]:
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(ttt[['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class']]).toarray())
# merge with main df on key values
ttt_OHE = ttt.join(enc_df)
ttt_OHE.drop(columns = ['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class'], inplace=True)

In [10]:
ttt_OHE.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [12]:
y_df = ttt_OHE[[27, 28]]

In [14]:
X_df = ttt_OHE.drop(columns = [27, 28])

In [15]:
X_df.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [13]:
y_df.head()

,27,28
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [18]:
from sklearn.preprocessing import StandardScaler

# create a scaler object
std_scaler = StandardScaler()
std_scaler
# fit and transform the data
X_std = pd.DataFrame(std_scaler.fit_transform(X_df))
y_std = pd.DataFrame(std_scaler.fit_transform(y_df))
X_std.head()


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,-0.52177,-0.733294,1.136603,-0.594228,-0.724898,1.238706,-0.52177,-0.733294,1.136603,-0.594228,...,-0.807294,-0.521770,-0.733294,1.136603,-0.594228,1.379504,-0.807294,-0.521770,1.363709,-0.879815
1,-0.52177,-0.733294,1.136603,-0.594228,-0.724898,1.238706,-0.52177,-0.733294,1.136603,-0.594228,...,-0.807294,-0.521770,1.363709,-0.879815,-0.594228,-0.724898,1.238706,-0.521770,1.363709,-0.879815
2,-0.52177,-0.733294,1.136603,-0.594228,-0.724898,1.238706,-0.52177,-0.733294,1.136603,-0.594228,...,-0.807294,-0.521770,1.363709,-0.879815,-0.594228,1.379504,-0.807294,-0.521770,-0.733294,1.136603
3,-0.52177,-0.733294,1.136603,-0.594228,-0.724898,1.238706,-0.52177,-0.733294,1.136603,-0.594228,...,-0.807294,-0.521770,1.363709,-0.879815,1.682855,-0.724898,-0.807294,1.916552,-0.733294,-0.879815
4,-0.52177,-0.733294,1.136603,-0.594228,-0.724898,1.238706,-0.52177,-0.733294,1.136603,-0.594228,...,-0.807294,1.916552,-0.733294,-0.879815,-0.594228,1.379504,-0.807294,1.916552,-0.733294,-0.879815


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [23]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [25]:
x_train, y_train, x_test, y_test = train_test_split(X_std, y_std)

In [39]:
x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
22,-0.52177,-0.733294,1.136603,-0.594228,-0.724898,1.238706,-0.52177,-0.733294,1.136603,-0.594228,...,1.238706,-0.52177,1.363709,-0.879815,-0.594228,-0.724898,1.238706,-0.521770,1.363709,-0.879815
229,-0.52177,-0.733294,1.136603,1.682855,-0.724898,-0.807294,-0.52177,-0.733294,1.136603,-0.594228,...,-0.807294,-0.52177,1.363709,-0.879815,1.682855,-0.724898,-0.807294,-0.521770,-0.733294,1.136603
830,-0.52177,1.363709,-0.879815,-0.594228,1.379504,-0.807294,-0.52177,1.363709,-0.879815,-0.594228,...,-0.807294,-0.52177,-0.733294,1.136603,1.682855,-0.724898,-0.807294,-0.521770,-0.733294,1.136603
332,-0.52177,1.363709,-0.879815,-0.594228,-0.724898,1.238706,-0.52177,1.363709,-0.879815,-0.594228,...,-0.807294,-0.52177,-0.733294,1.136603,-0.594228,-0.724898,1.238706,1.916552,-0.733294,-0.879815
647,-0.52177,-0.733294,1.136603,-0.594228,-0.724898,1.238706,-0.52177,1.363709,-0.879815,-0.594228,...,-0.807294,-0.52177,-0.733294,1.136603,-0.594228,-0.724898,1.238706,-0.521770,1.363709,-0.879815


In [35]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [36]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'],
              validation_data=(x_test, y_test))

In [38]:

model.fit(x_train, y_train, epochs=5)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: inputs=           0         1         2         3         4         5         6   \
22  -0.521770 -0.733294  1.136603 -0.594228 -0.724898  1.238706 -0.521770   
229 -0.521770 -0.733294  1.136603  1.682855 -0.724898 -0.807294 -0.521770   
830 -0.521770  1.363709 -0.879815 -0.594228  1.379504 -0.807294 -0.521770   
332 -0.521770  1.363709 -0.879815 -0.594228 -0.724898  1.238706 -0.521770   
647 -0.521770 -0.733294  1.136603 -0.594228 -0.724898  1.238706 -0.521770   
265 -0.521770 -0.733294  1.136603  1.682855 -0.724898 -0.807294 -0.521770   
744 -0.521770  1.363709 -0.879815 -0.594228 -0.724898  1.238706 -0.521770   
175 -0.521770 -0.733294  1.136603 -0.594228  1.379504 -0.807294 -0.521770   
344 -0.521770  1.363709 -0.879815 -0.594228 -0.724898  1.238706 -0.521770   
860 -0.521770  1.363709 -0.879815  1.682855 -0.724898 -0.807294 -0.521770   
565  1.916552 -0.733294 -0.879815 -0.594228  1.3

In [ ]:
val_loss, val_acc = model.evaluate(x_test, y_test)
print(val_loss, val_acc)

In [ ]:
model.save('tic-tac-toe.model')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
predictions = model.predict([x_test])

In [ ]:
'''This are probability distritutions. A tensor that need to be unrooled'''
print(predictions)

In [ ]:
import numpy as np

print(np.argmax(predictions[0]))

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here